In [ ]:
! pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
# Clone VisRec Tutorial repository
!git clone https://github.com/ialab-puc/VisualRecSys-Tutorial-IUI2021.git

%cd /content/VisualRecSys-Tutorial-IUI2021/

Cloning into 'VisualRecSys-Tutorial-IUI2021'...
remote: Enumerating objects: 479, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 479 (delta 99), reused 90 (delta 71), pack-reused 342
Receiving objects: 100% (479/479), 10.31 MiB | 24.33 MiB/s, done.
Resolving deltas: 100% (255/255), done.
/content/VisualRecSys-Tutorial-IUI2021


In [ ]:
import os

import numpy as np
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm.auto import tqdm

from datasets.utils import PreprocessingDataset
from models.utils import get_model_by_name
from utils.environment import modified_environ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Create image embeddings

In [ ]:
# Dataset
DATASET = "UGallery"
assert DATASET in ["UGallery", "Wikimedia"]

In [ ]:
# Parameters
BATCH_SIZE, NUM_WORKERS = 8, 4
IMAGES_EXT = ["*.gif", "*.jpg", "*.jpeg", "*.png", "*.webp"]
USE_GPU = True

# Model
MODEL = "resnet50"
LAYER = "" # if not defined the last layer, before the classification, output will be extracted
assert MODEL in ["alexnet", "vgg16", "resnet50"]

# Images path
IMAGES_DIR = None
if DATASET == "Wikimedia":
    IMAGES_DIR = os.path.join("/", "mnt", "data2", "wikimedia", "imagenes_tarea")
elif DATASET == "UGallery":
    IMAGES_DIR = '/content/drive/MyDrive/RecSysCuratorNet/images'


In [ ]:
# Paths (output)
LAYERED_OUTPUT = f"-{LAYER}" if LAYER else ""
OUTPUT_EMBEDDING_PATH = os.path.join("data", DATASET, f"embedding-{MODEL}{LAYERED_OUTPUT}.npy")


In [ ]:
import PIL
from PIL import ImageFile


# Needed for some images in the Wikimedia dataset
PIL.Image.MAX_IMAGE_PIXELS = 3_000_000_000
# Some images are "broken" in Wikimedia dataset
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [ ]:
IMAGES_DIR

'/content/drive/MyDrive/RecSysCuratorNet/images'

In [ ]:
%%time
# Setting up torch device (useful if GPU available)
print("\nCreating device...")
device = torch.device("cuda:0" if torch.cuda.is_available() and USE_GPU else "cpu")
if torch.cuda.is_available() != USE_GPU:
    print((f"\nNotice: Not using GPU - "
           f"Cuda available ({torch.cuda.is_available()}) "
           f"does not match USE_GPU ({USE_GPU})"
    ))

# Downloading models for feature extraction
print("\nDownloading model...")
with modified_environ(TORCH_HOME="."):
    print(f"Model: {MODEL} (pretrained on imagenet)")
    model = get_model_by_name(MODEL, output_layer=LAYER).eval().to(device)

# Setting up transforms and dataset
print("\nSetting up transforms and dataset...")
images_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
image_dataset = PreprocessingDataset(
    IMAGES_DIR,
    extensions=IMAGES_EXT,
    transform=images_transforms,
)
image_dataloader = DataLoader(image_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
print(f">> Images dataset: {len(image_dataset)}")

# Calculate embedding dimension size
dummy_input = torch.ones(1, *image_dataset[0]["image"].size()).to(device)
dummy_output = model(dummy_input)
emb_dim = dummy_output.shape[1:] if LAYER else dummy_output.size(1)
print(f">> Embedding dimension size: {emb_dim}")

# Feature extraction phase
print(f"\nFeature extraction...")
output_ids = np.empty(len(image_dataset), dtype=object)
if LAYER:
    output_embedding = torch.zeros((len(image_dataset), *emb_dim), dtype=torch.float32, device=device)
else:
    output_embedding = torch.zeros((len(image_dataset), emb_dim), dtype=torch.float32, device=device)

with torch.no_grad():
    for batch_i, sample in enumerate(tqdm(image_dataloader, desc="Feature extraction")):
        item_image = sample["image"].to(device)
        item_idx = sample["idx"]
        output_ids[[*item_idx]] = sample["id"]
        output_embedding[item_idx] = model(item_image).squeeze(-1).squeeze(-1)

output_embedding = output_embedding.cpu().numpy()

# Fill output embedding
embedding = np.ndarray(
    shape=(len(image_dataset), 2),
    dtype=object,
)
for i in range(len(image_dataset)):
    embedding[i] = np.asarray([output_ids[i], output_embedding[i]])
print(f">> Embedding shape: {embedding.shape}")

# Save embedding to file
print(f"\nSaving embedding to file... ({OUTPUT_EMBEDDING_PATH})")
np.save(OUTPUT_EMBEDDING_PATH, embedding, allow_pickle=True)

# Free some memory
if USE_GPU:
    print(f"\nCleaning GPU cache...")
    model = model.to(torch.device("cpu"))
    torch.cuda.empty_cache()

# Finished
print("\nDone")



Creating device...

Notice: Not using GPU - Cuda available (False) does not match USE_GPU (True)

Model: resnet50 (pretrained on imagenet)


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to ./hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]


Setting up transforms and dataset...


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> Images dataset: 13297
>> Embedding dimension size: 2048

Feature extraction...


Feature extraction:   0%|          | 0/1663 [00:00<?, ?it/s]

In [ ]:
a=np.load('/content/drive/MyDrive/RecSysCuratorNet/resnet_embeddings.npy',allow_pickle=True)

In [ ]:
emb_nw=dict(embedding)

In [ ]:
emb_of=dict(a)

In [ ]:
import numpy as np

In [ ]:
X_new=[]
X_of=[]

for l1 in tqdm(emb_nw.items()):
  key1=l1[0]
  value1=l1[1]
  j=9999999
  for l2 in emb_of.items():
    key2=l2[0]
    value2=l2[1]
    c=np.linalg.norm(value1-value2)
    c=np.round(c,decimals=0)
    if c<j:
      j=c
      k1=key1
      k2=key2
  X_new.append(k1)
  X_of.append(k2)


In [ ]:
trans_dict=dict(X_new,X_of)


In [ ]:
trans_dict.dump('ids_converted.npy')